## MODELS: latencia

In [ ]:
import time
import numpy as np
from ultralytics import YOLO
from pathlib import Path
import os

# Ruta a la carpeta con modelos exportados
projects_dir = Path("projects")

# Ruta a las imágenes de prueba
test_images = list(Path("data/test/images").glob("*.jpg"))[:50]
if not test_images:
    raise FileNotFoundError("No se encontraron imágenes en 'data/test/images'")

print(f"📷 Se usarán {len(test_images)} imágenes para la evaluación.\n")

# Crear lista para guardar resultados
results_list = []

# Iterar sobre cada carpeta de modelo
for model_dir in projects_dir.iterdir():
    weights_path = model_dir / "weights" / "best.pt"
    if not weights_path.exists():
        print(f"⚠️  No se encontró el modelo en: {weights_path}")
        continue

    print(f"🚀 Evaluando modelo: {model_dir.name}")

    try:
        model = YOLO(str(weights_path))

        # Medir latencias
        latencies = []
        for img_path in test_images:
            start = time.perf_counter()
            _ = model(img_path)
            end = time.perf_counter()
            latencies.append((end - start) * 1000)  # en milisegundos

        latency_p50 = np.percentile(latencies, 50)
        latency_p95 = np.percentile(latencies, 95)
        latency_mean = np.mean(latencies)

        results_list.append({
            "model": model_dir.name,
            "latency_p50_ms": round(latency_p50, 2),
            "latency_p95_ms": round(latency_p95, 2),
            "latency_mean_ms": round(latency_mean, 2)
        })

        print(f"   ✅ Latencia P50: {latency_p50:.2f} ms")
        print(f"   ✅ Latencia P95: {latency_p95:.2f} ms")
        print(f"   ✅ Latencia media: {latency_mean:.2f} ms\n")

    except Exception as e:
        print(f"❌ Error con el modelo {model_dir.name}: {e}\n")

# Mostrar resultados al final
import pandas as pd
df = pd.DataFrame(results_list)
df = df.sort_values(by="latency_p50_ms")
print("\n📊 Resumen de latencias:")
display(df)


## MODELS: val vs test

In [ ]:
import pandas as pd
from ultralytics import YOLO
from pathlib import Path
import json

# ================================================================
# CONFIGURACIÓN
# ================================================================
projects_dir = Path("projects")

# Detectar data.yaml
default_yaml = Path("data.yaml")

if default_yaml.exists():
    data_yaml = str(default_yaml)
else:
    data_folder = Path("data")
    yaml_files = list(data_folder.glob("*.yaml")) + list(data_folder.glob("*.yml"))

    if len(yaml_files) == 0:
        raise FileNotFoundError("No se encontró 'data.yaml' ni ningún archivo YAML dentro de 'data/'.")
    
    data_yaml = str(yaml_files[0])
    print(f"⚠️ 'data.yaml' no encontrado. Usando: {data_yaml}")

# ================================================================
# DETECTAR MODELOS AUTOMÁTICAMENTE
# ================================================================
model_paths = list(projects_dir.glob("*/weights/best.pt"))

if not model_paths:
    raise FileNotFoundError("No se encontraron modelos en projects/*/weights/best.pt")

print("📦 Modelos detectados:")
for p in model_paths:
    print("  -", p)


results_global = []  # Para el CSV final

# ================================================================
# PROCESAR CADA MODELO
# ================================================================
for model_path in model_paths:

    model_name = model_path.parent.parent.name
    print(f"\n==============================")
    print(f"🚀 Evaluando modelo: {model_name}")
    print(f"==============================")

    output_dir = Path(f"results/{model_name}")
    output_dir.mkdir(parents=True, exist_ok=True)

    model = YOLO(str(model_path))

    # ------------------------ VAL SET ------------------------
    print("📊 Ejecutando evaluación en VAL set...")
    metrics_val = model.val(data=data_yaml, split="val")
    val_dict = metrics_val.results_dict
    val_dict = {f"val_{k.replace('metrics/', '').replace('(B)', '')}": v for k, v in val_dict.items()}

    # Guardar JSON de val
    with open(output_dir / f"{model_name}_val.json", "w") as f:
        json.dump(val_dict, f, indent=4)

    # ------------------------ TEST SET ------------------------
    print("📊 Ejecutando evaluación en TEST set...")
    metrics_test = model.val(data=data_yaml, split="test")
    test_dict = metrics_test.results_dict
    test_dict = {f"test_{k.replace('metrics/', '').replace('(B)', '')}": v for k, v in test_dict.items()}

    # Guardar JSON de test
    with open(output_dir / f"{model_name}_test.json", "w") as f:
        json.dump(test_dict, f, indent=4)

    # ------------------------ COMBINAR VAL + TEST ------------------------
    combined = {"model": model_name}
    combined.update(val_dict)
    combined.update(test_dict)

    # Guardar CSV del modelo
    pd.DataFrame([combined]).to_csv(output_dir / f"{model_name}_val_test.csv", index=False)

    # Añadir al global
    results_global.append(combined)


# ================================================================
# CREAR CSV GLOBAL
# ================================================================
df = pd.DataFrame(results_global)
df = df.set_index("model", drop=False)

df.to_csv("results/VAL_TEST_COMPARISON.csv", index=False)

print("\n📌 COMPARATIVA FINAL VAL vs TEST")
print(df)

print("\n📄 CSV global guardado en: results/VAL_TEST_COMPARISON.csv")
